In [1]:
import sys
# sys.path is a list of absolute path strings
sys.path.append('/mnt/sda/boris_stanoev/PropStar/')

In [2]:
from normalization.normalize import Normalize
from sklearn.datasets import fetch_openml

In [3]:
data = fetch_openml(data_id=1558)
denormalized_table = data.data.join(data.target)

/mnt/sda/boris_stanoev/PropStar/venv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
denormalized_table.columns.values

array(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'Class'], dtype=object)

In [6]:
denormalized_table.rename(columns={"V1": "age",
                                   "V2": "job",
                                   "V3": "marital",
                                   "V4": "education",
                                   "V5": "default",
                                   "V6": "balance",
                                   "V7": "housing",
                                   "V8": "loan",
                                   "V9": "contact",
                                   "V10": "day",
                                   "V11": "month",
                                   "V12": "duration",
                                   "V13": "campaign",
                                   "V14": "pdays",
                                   "V15": "previous",
                                   "V16": "poutcome",
                                  }, inplace=True)

In [ ]:
"""
bank client data:
    1 - age (numeric)
    
    2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur", "student","blue-collar","self-employed","retired","technician","services")
    
    3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)
    
    4 - education (categorical: "unknown","secondary","primary","tertiary")
    
    5 - default: has credit in default? (binary: "yes","no")
    
    6 - balance: average yearly balance, in euros (numeric)
    
    7 - housing: has housing loan? (binary: "yes","no")
    
    8 - loan: has personal loan? (binary: "yes","no")

related with the last contact of the current campaign:
    9 - contact: contact communication type (categorical: "unknown","telephone","cellular")
    
    10 - day: last contact day of the month (numeric)
    
    11 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
    
    12 - duration: last contact duration, in seconds (numeric)

other attributes:
    13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
    
    14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
    
    15 - previous: number of contacts performed before this campaign and for this client (numeric)
    
    16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

output variable (desired target):
17 - y - has the client subscribed a term deposit? (binary: "yes","no")

"""

In [7]:
denormalized_table

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Class
0,30.0,unemployed,married,primary,no,1787.0,no,no,cellular,19.0,oct,79.0,1.0,-1.0,0.0,unknown,1
1,33.0,services,married,secondary,no,4789.0,yes,yes,cellular,11.0,may,220.0,1.0,339.0,4.0,failure,1
2,35.0,management,single,tertiary,no,1350.0,yes,no,cellular,16.0,apr,185.0,1.0,330.0,1.0,failure,1
3,30.0,management,married,tertiary,no,1476.0,yes,yes,unknown,3.0,jun,199.0,4.0,-1.0,0.0,unknown,1
4,59.0,blue-collar,married,secondary,no,0.0,yes,no,unknown,5.0,may,226.0,1.0,-1.0,0.0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33.0,services,married,secondary,no,-333.0,yes,no,cellular,30.0,jul,329.0,5.0,-1.0,0.0,unknown,1
4517,57.0,self-employed,married,tertiary,yes,-3313.0,yes,yes,unknown,9.0,may,153.0,1.0,-1.0,0.0,unknown,1
4518,57.0,technician,married,secondary,no,295.0,no,no,cellular,19.0,aug,151.0,11.0,-1.0,0.0,unknown,1
4519,28.0,blue-collar,married,secondary,no,1137.0,no,no,cellular,6.0,feb,129.0,4.0,211.0,3.0,other,1


In [16]:
norm = Normalize(denormalized_table=denormalized_table, denormalized_table_name="target")

In [17]:
client_cols = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan']
norm.create_entity(source_entity_name="target", target_entity_name="client", columns=client_cols,
                   distinct_values=True)

In [18]:
contact_cols = ['contact', 'day', 'month', 'duration']
norm.create_entity(source_entity_name="target", target_entity_name="contact", columns=contact_cols,
                   distinct_values=True)

In [19]:
attribute_cols = ['campaign', 'pdays', 'previous', 'poutcome']
norm.create_entity(source_entity_name="target", target_entity_name="attribute", columns=attribute_cols,
                   distinct_values=True)

In [20]:
norm.primary_keys
norm.relations
norm.entity_set.keys()


dict_keys(['target', 'client', 'contact', 'attribute'])

In [21]:
for entity in norm.entity_set.keys():
    print(
        f"ALTER TABLE {entity} ADD CONSTRAINT PK_{entity} PRIMARY KEY ({norm.primary_keys[entity]});")

for parent_table in norm.relations:
    for child_table in norm.relations[parent_table]:
        print(f"ALTER TABLE {child_table} ADD CONSTRAINT FK_{child_table}_{parent_table} "
              f"FOREIGN KEY ({norm.relations[parent_table][child_table][1]}) "
              f"REFERENCES {parent_table}({norm.relations[parent_table][child_table][0]}));")


ALTER TABLE target ADD CONSTRAINT PK_target PRIMARY KEY (id);
ALTER TABLE client ADD CONSTRAINT PK_client PRIMARY KEY (client_id);
ALTER TABLE contact ADD CONSTRAINT PK_contact PRIMARY KEY (contact_id);
ALTER TABLE attribute ADD CONSTRAINT PK_attribute PRIMARY KEY (attribute_id);
ALTER TABLE target ADD CONSTRAINT FK_target_client FOREIGN KEY (client_id) REFERENCES client(client_id));
ALTER TABLE target ADD CONSTRAINT FK_target_contact FOREIGN KEY (contact_id) REFERENCES contact(contact_id));
ALTER TABLE target ADD CONSTRAINT FK_target_attribute FOREIGN KEY (attribute_id) REFERENCES attribute(attribute_id));


In [23]:
norm.persist_entity_set(database_name="bank_marketing_exp1")

2023-09-18 21:47:48,024 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2023-09-18 21:47:48,026 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-18 21:47:48,029 INFO sqlalchemy.engine.Engine SELECT schema_name()
2023-09-18 21:47:48,030 INFO sqlalchemy.engine.Engine [generated in 0.00120s] ()
2023-09-18 21:47:48,035 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2023-09-18 21:47:48,036 INFO sqlalchemy.engine.Engine [generated in 0.00112s] ()
2023-09-18 21:47:48,039 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2023-09-18 21:47:48,040 INFO sqlalchemy.engine.Engine [generated in 0.00128s] ()
2023-09-18 21:47:48,044 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-18 21:47:48,045 INFO sqlalchemy.engine.Engine IF (SCHEMA_ID('bank_marketing_exp1') IS NOT NULL) BEGIN DROP SCHEMA [bank_marketing_exp1]; END 
2023-09-18 21:47:48,046 I